In [9]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

### Considerar tener la ruta de la siguiente manera

In [2]:
carpeta = '../datasets/yellow'

# Lista para almacenar los DataFrames de cada archivo
dfs = []

# Iterar sobre los archivos en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith('.parquet'):
        ruta_archivo = os.path.join(carpeta, archivo)
        # Leer el archivo Parquet y añadirlo a la lista de DataFrames
        df = pd.read_parquet(ruta_archivo)
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
df_yellow = pd.concat(dfs, ignore_index=True)

### Columnas a eliminar, porque no influyen en los objetivos planteados en el proyecto

In [3]:
df_yellow.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'Airport_fee'],
      dtype='object')

In [4]:
# Eliminamos columnas
columnas_a_eliminar = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 
                       'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee', 'Airport_fee']
df_yellow.drop(columns=columnas_a_eliminar, axis=1, inplace=True)

In [5]:
# Vemos las dimensiones de nuestro DF
df_yellow.shape

(108870632, 10)

In [6]:
# Eliminamos los registros que tengan nulos
df_sin_nulos = df_yellow.dropna()

In [7]:
# Verficamos las dimensiones, para saber como quedaron luego de eliminar los nulos
df_sin_nulos.shape

(104714278, 10)

### Convertimos las columnas con números enteros a `formato INT32`, para optimizar espacio

In [8]:
columns_to_int32 = ['VendorID', 'passenger_count', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type']

In [10]:
df_sin_nulos[columns_to_int32] = df_sin_nulos[columns_to_int32].astype('int32')
df_sin_nulos.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                   int32
trip_distance                   float64
RatecodeID                        int32
PULocationID                      int32
DOLocationID                      int32
payment_type                      int32
total_amount                    float64
dtype: object

### Eliminación datos que salen de los `limites` que establece el `diccionario`

In [11]:
# Eliminamos los datos superiores a 6, porque el diccionario establece solo datos del 1-6
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['RatecodeID'] > 6].index, inplace=True)

In [12]:
# Comprobamos los datos que tiene la columna RatecodeID
df_sin_nulos['RatecodeID'].unique()

array([1, 2, 4, 5, 3, 6])

In [13]:
# Establecemos los límites para la columnas de tiempo, puesto que solo estamos usando datos de 2021 hasta 2023
df_sin_nulos = df_sin_nulos[(df_sin_nulos['tpep_pickup_datetime'].dt.year >= 2021) & (df_sin_nulos['tpep_pickup_datetime'].dt.year <= 2023)]
# Establecemos el límite superior en 2024, por los servicios tomados en 2023-12-31 y terminaron en el año siguiente
df_sin_nulos = df_sin_nulos[(df_sin_nulos['tpep_dropoff_datetime'].dt.year >= 2021) & (df_sin_nulos['tpep_dropoff_datetime'].dt.year <= 2024)]

In [14]:
# Verificamos que los límites esten correctamente
print(df_sin_nulos['tpep_pickup_datetime'].max())
print(df_sin_nulos['tpep_pickup_datetime'].min())
print(df_sin_nulos['tpep_dropoff_datetime'].max())
print(df_sin_nulos['tpep_dropoff_datetime'].min())

2023-12-31 23:59:59
2021-01-01 00:00:04
2024-01-02 20:13:25
2021-01-01 00:01:40


### Creacipon de columna, que establece la hora que se realizo el servicio

In [15]:
df_sin_nulos['pickup_hour'] = df_sin_nulos['tpep_pickup_datetime'].dt.hour

In [16]:
df_sin_nulos.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,pickup_hour
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,142,43,2,11.80,0
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,238,151,2,4.30,0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,132,165,1,51.95,0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,138,132,1,36.35,0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,68,33,1,24.36,0


### Tratamiento de variables temporales `tpep_dropoff_datetime`

In [17]:
# Creación de columna 'duracion_viaje' para tener una referencia concreta de la duración de cada servicio
df_sin_nulos['duracion_viaje'] = (df_sin_nulos['tpep_dropoff_datetime'] - df_sin_nulos['tpep_pickup_datetime']).dt.total_seconds()

In [18]:
df_sin_nulos.shape

(104354541, 12)

In [19]:
df_sin_nulos.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,pickup_hour,duracion_viaje
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,142,43,2,11.80,0,362.0
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,238,151,2,4.30,0,59.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,132,165,1,51.95,0,1656.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,138,132,1,36.35,0,913.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,68,33,1,24.36,0,992.0


In [20]:
# Creación de columnas que tengan información por separado de la fecha y hora
df_sin_nulos['pickup_date'] = df_sin_nulos['tpep_pickup_datetime'].dt.date # Fecha
df_sin_nulos['pickup_time'] = df_sin_nulos['tpep_pickup_datetime'].dt.time # Hora
df_sin_nulos['dropoff_date'] = df_sin_nulos['tpep_dropoff_datetime'].dt.date # Fecha
df_sin_nulos['dropoff_time'] = df_sin_nulos['tpep_dropoff_datetime'].dt.time # Hora

In [21]:
# Eliminar las columnas que tienen la información unificada de Fecha y Hora
df_sin_nulos.drop(columns=['tpep_pickup_datetime'], inplace=True)
df_sin_nulos.drop(columns=['tpep_dropoff_datetime'], inplace=True)

In [22]:
# Verificamos la estructura nueva, luego de crear y eliminar columnas
df_sin_nulos.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,pickup_hour,duracion_viaje,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,1,2.10,1,142,43,2,11.80,0,362.0,2021-01-01,00:30:10,2021-01-01,00:36:12
1,1,1,0.20,1,238,151,2,4.30,0,59.0,2021-01-01,00:51:20,2021-01-01,00:52:19
2,1,1,14.70,1,132,165,1,51.95,0,1656.0,2021-01-01,00:43:30,2021-01-01,01:11:06
3,1,0,10.60,1,138,132,1,36.35,0,913.0,2021-01-01,00:15:48,2021-01-01,00:31:01
4,2,1,4.94,1,68,33,1,24.36,0,992.0,2021-01-01,00:31:49,2021-01-01,00:48:21


### Eliminación de columnas que se considera innecesario para los siguientes procesos

In [23]:
df_sin_nulos.drop(columns=['pickup_time', 'dropoff_time', 'dropoff_date'], inplace=True)

In [24]:
# Revisamos los formatos de cada columna
df_sin_nulos.dtypes

VendorID             int32
passenger_count      int32
trip_distance      float64
RatecodeID           int32
PULocationID         int32
DOLocationID         int32
payment_type         int32
total_amount       float64
pickup_hour          int32
duracion_viaje     float64
pickup_date         object
dtype: object

In [25]:
# Convertimos columnas 'pickup_date' y 'duracion_viaje' a formato DATE e INT32, para optimizar peso
df_sin_nulos['pickup_date'] = pd.to_datetime(df_sin_nulos['pickup_date'])
df_sin_nulos['duracion_viaje'] = df_sin_nulos['duracion_viaje'].astype('int32')

In [26]:
df_sin_nulos.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,pickup_hour,duracion_viaje,pickup_date
0,1,1,2.10,1,142,43,2,11.80,0,362,2021-01-01
1,1,1,0.20,1,238,151,2,4.30,0,59,2021-01-01
2,1,1,14.70,1,132,165,1,51.95,0,1656,2021-01-01
3,1,0,10.60,1,138,132,1,36.35,0,913,2021-01-01
4,2,1,4.94,1,68,33,1,24.36,0,992,2021-01-01


### Tratamiento columna `passenger_count`

In [27]:
# Revisamos los registros únicos para la columna 'passenger_count'
df_sin_nulos['passenger_count'].unique()

array([  1,   0,   2,   3,   5,   4,   6,   8,   7,   9, 112,  96])

In [28]:
# Eliminación de registros que están fuera de la normativa legal para los taxis amarillos en EEUU
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['passenger_count'] == 0].index, inplace=True)
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['passenger_count'] > 4].index, inplace=True)

In [29]:
# Corroboramos los cambios aplicados
df_sin_nulos['passenger_count'].unique()

array([1, 2, 3, 4])

In [30]:
# Revisamos las dimensiones del DF, luego de los cambios efectuados
df_sin_nulos.shape

(99189448, 11)

### Tratamiento de columnas `total_amount` y `duracion_viaje`

In [31]:
# Calcular la media, mediana, desviación estándar, mínimo y máximo de la columna:  'total_amount'
mean_seconds = df_sin_nulos['total_amount'].mean()
median_seconds = df_sin_nulos['total_amount'].median()
std_seconds = df_sin_nulos['total_amount'].std()
min_seconds = df_sin_nulos['total_amount'].min()
max_seconds = df_sin_nulos['total_amount'].max()

# Revisar los resultados
print("Minimo:", min_seconds)
print("Maximo:", max_seconds)
print("Media:", mean_seconds)
print("Mediana:", median_seconds)
print("Desviación estándar:", std_seconds)

Minimo: -2567.8
Maximo: 818286.74
Media: 23.27464075110078
Mediana: 17.16
Desviación estándar: 125.06789931294153


In [32]:
# Calcular la media, mediana, desviación estándar, mínimo y máximo de la columna:  'duracion_viaje'
mean_seconds = df_sin_nulos['duracion_viaje'].mean()
median_seconds = df_sin_nulos['duracion_viaje'].median()
std_seconds = df_sin_nulos['duracion_viaje'].std()
min_seconds = df_sin_nulos['duracion_viaje'].min()
max_seconds = df_sin_nulos['duracion_viaje'].max()

# Revisar los resultados
print("Minimo:", min_seconds)
print("Maximo:", max_seconds)
print("Media:", mean_seconds)
print("Mediana:", median_seconds)
print("Desviación estándar:", std_seconds)

Minimo: -6268355
Maximo: 1760191
Media: 1012.1953548728288
Mediana: 711.0
Desviación estándar: 3052.235520186167


In [33]:
# Calcular los cuartiles para las columnas 'total_amount' y 'duracion_viaje'
cuartiles_total_amount = np.percentile(df_sin_nulos['total_amount'], [10, 50, 90])
cuartiles_duracion_viaje = np.percentile(df_sin_nulos['duracion_viaje'], [5, 50, 95])
print(cuartiles_total_amount)
print(cuartiles_duracion_viaje)

[10.3  17.16 45.  ]
[ 194.  711. 2395.]


In [34]:
# Eliminamos registros basandonos en los cuartiles identificados en el paso anterior
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['total_amount'] < 10.3 ].index, inplace=True)
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['total_amount'] > 44.35 ].index, inplace=True)
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['duracion_viaje'] < 208 ].index, inplace=True)
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['duracion_viaje'] > 2393 ].index, inplace=True)

In [35]:
# Revisamos las dimensiones del DF, luego de eliminar una cantidad importante de registros
df_sin_nulos.shape

(77666859, 11)

### Tratamiento a columna `trip_distance`

In [36]:
# Contar los registros con duración de viaje inferior a 0.5
registros_inferiores_a_cero = df_sin_nulos[df_sin_nulos['trip_distance'] < 0.5]
cantidad_registros_inferiores_a_cero = len(registros_inferiores_a_cero)
print("Cantidad de registros con trip_distance inferior a 0.5:", cantidad_registros_inferiores_a_cero)

Cantidad de registros con trip_distance inferior a 0.5: 927409


In [37]:
# Contar los registros con duración de viaje superiores a 13
registros_inferiores_a_cero = df_sin_nulos[df_sin_nulos['trip_distance'] > 13]
cantidad_registros_inferiores_a_cero = len(registros_inferiores_a_cero)
print("Cantidad de registros con trip_distance superior a 13:", cantidad_registros_inferiores_a_cero)

Cantidad de registros con trip_distance superior a 13: 69056


In [38]:
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['trip_distance'] < 0.5 ].index, inplace=True)
df_sin_nulos.drop(df_sin_nulos[df_sin_nulos['trip_distance'] > 13 ].index, inplace=True)

In [39]:
df_sin_nulos.shape

(76670394, 11)

In [40]:
df_sin_nulos.dtypes

VendorID                    int32
passenger_count             int32
trip_distance             float64
RatecodeID                  int32
PULocationID                int32
DOLocationID                int32
payment_type                int32
total_amount              float64
pickup_hour                 int32
duracion_viaje              int32
pickup_date        datetime64[ns]
dtype: object

In [41]:
df_sin_nulos.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,pickup_hour,duracion_viaje,pickup_date
0,1,1,2.10,1,142,43,2,11.80,0,362,2021-01-01
4,2,1,4.94,1,68,33,1,24.36,0,992,2021-01-01
5,1,1,1.60,1,224,68,1,14.15,0,481,2021-01-01
6,1,1,4.10,1,95,157,2,17.30,0,1020,2021-01-01
7,1,1,5.70,1,90,40,2,21.80,0,1085,2021-01-01


In [42]:
df_sin_nulos.to_parquet('../datasets/yellow_final_documentado.parquet', compression='gzip')